In [11]:
import os
import cv2

# Paths for input and output folders
input_folder = '/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/qumran_segmentation_dataset/normalized_last_band_images'
output_folder = '/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/otsu_masks'

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Iterate through each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg'):
        # Construct the full input file path
        input_path = os.path.join(input_folder, filename)

        # Read the grayscale image
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        # Apply Otsu's thresholding
        _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Construct the output file path
        output_path = os.path.join(output_folder, filename[:7]+'.png')

        # Save the binarized image
        cv2.imwrite(output_path, binary_image)

print("Otsu binarization completed and results saved.")


Otsu binarization completed and results saved.


In [1]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.filters import threshold_sauvola

# Paths
input_folder = "/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/qumran_segmentation_dataset/normalized_last_band_images"
output_folder = "/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/sauvola_masks"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define tile size to split images into smaller manageable pieces
tile_size = 500  # Adjust the tile size to control memory usage

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg'):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = io.imread(image_path)

        # Create an empty output image
        binary_image = np.zeros_like(image, dtype=np.uint8)

        # Split the image into tiles and process each one
        for y in range(0, image.shape[0], tile_size):
            for x in range(0, image.shape[1], tile_size):
                # Define the tile boundaries
                y_end = min(y + tile_size, image.shape[0])
                x_end = min(x + tile_size, image.shape[1])

                # Extract the tile
                tile = image[y:y_end, x:x_end]

                # Apply Sauvola threshold to the tile
                window_size = 125  # You can adjust the window size if needed
                thresh_sauvola = threshold_sauvola(tile, window_size=window_size)
                tile_binary = tile > thresh_sauvola

                # Convert the boolean mask to uint8 (0 or 255)
                binary_image[y:y_end, x:x_end] = (tile_binary * 255).astype(np.uint8)

        # Save the result
        output_path = os.path.join(output_folder, filename[:7]+'.png')
        cv2.imwrite(output_path, binary_image)

print("Sauvola binarization completed for all images.")



Sauvola binarization completed for all images.


In [2]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".bmp"):
            path = os.path.join(folder, filename)
            images[filename] = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return images

def and_otsu_and_sauvola(otsu_folder, sauvola_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    otsu_masks = load_images_from_folder(otsu_folder)
    sauvola_masks = load_images_from_folder(sauvola_folder)

    for filename, otsu_mask in otsu_masks.items():
        if filename in sauvola_masks:
            sauvola_mask = sauvola_masks[filename]
            # Ensure the masks are binarized (0 or 255)
            _, otsu_mask = cv2.threshold(otsu_mask, 127, 255, cv2.THRESH_BINARY)
            _, sauvola_mask = cv2.threshold(sauvola_mask, 127, 255, cv2.THRESH_BINARY)

            # Perform OR operation
            combined_mask = cv2.bitwise_and(otsu_mask, sauvola_mask)

            # Save the result
            output_path = os.path.join(output_folder, filename[:7]+'.png')
            cv2.imwrite(output_path, combined_mask)

otsu_folder = "/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/otsu_masks"
sauvola_folder = "/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/sauvola_masks"
output_folder = "/content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/otsu_and_sauvola_masks"


and_otsu_and_sauvola(otsu_folder, sauvola_folder, output_folder)

print(f"Combined masks have been saved to {output_folder}")


Combined masks have been saved to /content/drive/MyDrive/Colab Notebooks/icdar_ijdar_2025/msem_results/otsu_and_sauvola_masks
